In [13]:
import torch
import numpy as np

# complex valued tensor class
from cplxmodule import cplx

# converters
from cplxmodule.nn import RealToCplx, CplxToReal, CplxToCplx

# layers of encapsulating other complex valued layers
from cplxmodule.nn import CplxSequential

# common layers
from cplxmodule.nn import CplxConv1d, CplxLinear

# activation layers
from cplxmodule.nn import CplxModReLU

from AlexNet_Complex import AlexNet_Complex

In [3]:
n_features, n_channels = 16, 4
z = torch.randn(3, n_features*2)

cplx = RealToCplx()(z)
print(cplx)

Cplx(
  real=tensor([[ 0.6067, -1.2986,  1.1366,  0.9511,  1.3872,  0.7579,  0.1695, -0.4848,
          0.4126, -0.4437, -1.4575, -0.4797, -0.6389, -0.6942,  0.6718, -0.0090],
        [-0.3511, -1.2464, -1.1513,  0.6919, -0.1823, -1.7045,  0.7023,  0.4357,
          0.2007,  0.0655, -1.8841,  1.2354,  1.2319, -1.6471, -0.1590,  0.0102],
        [ 0.6116, -1.0051, -2.1569, -0.2353, -2.8190, -0.4525, -0.4422,  1.1152,
         -1.4332,  0.9385, -2.3541,  1.2192, -0.9261,  0.9732, -1.4989,  0.8713]]),
  imag=tensor([[-0.1520,  0.7388,  0.6029,  2.0101, -1.0517,  0.1079,  0.0452,  0.9122,
          0.9120,  0.0364,  0.4965,  0.5173,  0.5066,  1.0062,  0.9033, -0.5244],
        [ 0.4677, -0.9746,  0.1603,  0.7642,  0.2519, -1.2987,  1.1950,  0.1669,
          0.4178,  0.1939,  0.3925, -0.4821, -1.4808, -0.4477, -0.7293, -1.8995],
        [ 0.4768, -0.3098,  0.4167, -1.1880, -0.1842,  0.7276,  0.8839, -0.8158,
          1.1323, -0.3404, -0.1977, -0.7829, -0.5455, -0.5593,  1.1688,  0.2307]])

In [5]:
z.shape
# batch 3, channel 4

torch.Size([3, 32])

tensor([[ 0.6067, -0.1520, -1.2986,  0.7388,  1.1366,  0.6029,  0.9511,  2.0101,
          1.3872, -1.0517,  0.7579,  0.1079,  0.1695,  0.0452, -0.4848,  0.9122,
          0.4126,  0.9120, -0.4437,  0.0364, -1.4575,  0.4965, -0.4797,  0.5173,
         -0.6389,  0.5066, -0.6942,  1.0062,  0.6718,  0.9033, -0.0090, -0.5244],
        [-0.3511,  0.4677, -1.2464, -0.9746, -1.1513,  0.1603,  0.6919,  0.7642,
         -0.1823,  0.2519, -1.7045, -1.2987,  0.7023,  1.1950,  0.4357,  0.1669,
          0.2007,  0.4178,  0.0655,  0.1939, -1.8841,  0.3925,  1.2354, -0.4821,
          1.2319, -1.4808, -1.6471, -0.4477, -0.1590, -0.7293,  0.0102, -1.8995],
        [ 0.6116,  0.4768, -1.0051, -0.3098, -2.1569,  0.4167, -0.2353, -1.1880,
         -2.8190, -0.1842, -0.4525,  0.7276, -0.4422,  0.8839,  1.1152, -0.8158,
         -1.4332,  1.1323,  0.9385, -0.3404, -2.3541, -0.1977,  1.2192, -0.7829,
         -0.9261, -0.5455,  0.9732, -0.5593, -1.4989,  1.1688,  0.8713,  0.2307]])

In [10]:
n_features, n_channels = 16, 4
z2 = torch.randn(256, n_channels, n_features * 2)

complex_model = CplxSequential(

    CplxLinear(n_features, n_features, bias=True),

    # complex: batch x n_channels x n_features
    CplxConv1d(n_channels, 3 * n_channels, kernel_size=4, stride=1, bias=False),

    # complex: batch x (3 * n_channels) x (n_features - (4-1))
    CplxModReLU(threshold=0.15),

    # complex: batch x (3 * n_channels) x (n_features - (4-1))
    CplxToCplx[torch.nn.Flatten](start_dim=-2),

    CplxToCplx[torch.tanh](),
    
)

real_input_model = torch.nn.Sequential(
    
    # real: batch x (n_features * 2)
    RealToCplx(),

    # complex: batch x n_features
    complex_model,

    # complex: batch x (3 * n_channels * (n_features - (4-1)))
    CplxToReal(),

    # real: batch x ((3 * n_channels * (n_features - (4-1))) * 2)
)

In [11]:
print(real_input_model(z2).shape)


torch.Size([256, 312])


In [39]:
# Mine: [batch, signal, real/compl]
# this case: [2, 6, 2]

# Dummy would be z2 bc stacking real/imag
z2 = torch.randn(2, 12)

In [41]:
print(z2)
cplx = RealToCplx()(z2)
print(cplx)

tensor([[ 0.0681, -0.5854, -1.4799,  1.6354,  1.6686, -0.3169,  0.2735,  1.4125,
          1.9294,  0.0650, -0.0268, -0.5446],
        [ 1.3389,  0.6516, -0.6153, -0.3866,  0.0661, -0.6029,  0.1224,  1.2243,
         -0.4818,  0.4872,  2.5807, -1.8771]])
Cplx(
  real=tensor([[ 0.0681, -1.4799,  1.6686,  0.2735,  1.9294, -0.0268],
        [ 1.3389, -0.6153,  0.0661,  0.1224, -0.4818,  2.5807]]),
  imag=tensor([[-0.5854,  1.6354, -0.3169,  1.4125,  0.0650, -0.5446],
        [ 0.6516, -0.3866, -0.6029,  1.2243,  0.4872, -1.8771]])
)


In [16]:
a = np.array([[1, 2, 3]])
b = np.array([[[4, 5, 6]]])
np.dstack((a,b)).flatten()

array([1, 2, 3, 4, 5, 6])

In [18]:
A = np.array([[[1, 2, 3]]])
B = np.array([[[4, 5, 6]]])
np.vstack((A,B)).ravel('F')

array([1, 4, 2, 5, 3, 6])

In [59]:
start = np.random.random((2,2,6))
print(start[:,0,:])
print(start[:,1,:])

[[0.44106694 0.32125723 0.71886483 0.83224281 0.7731201  0.57266095]
 [0.59767141 0.34940249 0.78568625 0.24294884 0.51635719 0.27304299]]
[[0.54909248 0.76247947 0.06956216 0.77578655 0.37675704 0.18509387]
 [0.38820964 0.66273056 0.18451362 0.56637537 0.54899748 0.74475515]]


In [60]:
print(start)

[[[0.44106694 0.32125723 0.71886483 0.83224281 0.7731201  0.57266095]
  [0.54909248 0.76247947 0.06956216 0.77578655 0.37675704 0.18509387]]

 [[0.59767141 0.34940249 0.78568625 0.24294884 0.51635719 0.27304299]
  [0.38820964 0.66273056 0.18451362 0.56637537 0.54899748 0.74475515]]]


In [61]:
start.reshape(2,-1)

array([[0.44106694, 0.32125723, 0.71886483, 0.83224281, 0.7731201 ,
        0.57266095, 0.54909248, 0.76247947, 0.06956216, 0.77578655,
        0.37675704, 0.18509387],
       [0.59767141, 0.34940249, 0.78568625, 0.24294884, 0.51635719,
        0.27304299, 0.38820964, 0.66273056, 0.18451362, 0.56637537,
        0.54899748, 0.74475515]])

In [53]:
np.vstack((start[0,:,0],start[0,:,1])).ravel('F')

array([0.8609836 , 0.61582542, 0.26848256, 0.81180861, 0.48528936,
       0.23499443, 0.5135409 , 0.6870821 , 0.61568539, 0.69028395,
       0.83473922, 0.5500271 ])

In [32]:
#from cusignal import chirp, spectrogram
import numpy as np
# Bandwidt
from numpy import random
#from LFMSignal import *
from scipy.signal import chirp, spectrogram, hilbert
import matplotlib.pyplot as plt

# Dataset consts
seconds = pw = 50e-6
b0, b1 = 4e6, 6e6-2e5
fs = 2 * 8e6
classes = 10
bandwidths = np.linspace(b0, b1, num=classes)

# Create labels and preallocate numpy array
"""

"""
dataset = np.zeros((int(1e5), 800, 2), dtype=np.float32)
labels = np.zeros((10000)) # 1e4 x 10 classes
for i in range(1,10):
    labels = np.concatenate((labels, np.ones((10000))*i))
labels = labels.astype(np.int64)

class_samples = 10000
# Iter classes
for label in range(classes):
    #number of samples per class
    for j in range(class_samples):

        idx = j + class_samples*label 
        #if idx % 1000 == 0: print("Sample: {} - {}%".format(idx, 100*idx/(class_samples*classes)), end=" ")
        seconds = pw = 50e-6
        fs = 2 * 8e6
        f0 = random.randint(0, 2e6)
        f1 = f0 + bandwidths[label]
        t = np.linspace(0, seconds, seconds * fs, endpoint=False)
        data = chirp(t, f0=f0, f1=f1, t1=seconds, method='linear')
        print(hilbert(data).shape)
        dataset[idx] = hilbert(data).view('(2,)float').astype('float32')
        print(dataset[idx])
        print(dataset[idx].shape)
        print(dataset[idx][0][0])
        print(dataset[idx][0][1])
        break
    break

(800,)
[[ 1.         -0.6183733 ]
 [ 0.98617315 -0.19323896]
 [ 0.9444313   0.10218484]
 ...
 [ 0.03088558  0.79434407]
 [-0.99160004 -0.48234573]
 [ 0.28812164 -1.3582546 ]]
(800, 2)
1.0
-0.6183733


In [63]:
import cplxmodule.nn as cplxnn

complex_backbone = cplxnn.CplxSequential(
    #cplxnn.CplxAdaptiveModReLU()
    cplxnn.CplxConv1d(1, 96, kernel_size=11, stride=4, padding=1), cplxnn.CplxModReLU(),
    cplxnn.CplxConv1d(96, 96, kernel_size=3, padding=1, stride=2), cplxnn.CplxModReLU(), # In place of pooling
    cplxnn.CplxConv1d(96, 256, kernel_size=5, padding=2), cplxnn.CplxModReLU(),
    cplxnn.CplxConv1d(256, 256, kernel_size=3, padding=1, stride=2), cplxnn.CplxModReLU(),
    cplxnn.CplxConv1d(256, 384, kernel_size=3, padding=1), cplxnn.CplxModReLU(),
    cplxnn.CplxConv1d(384, 384, kernel_size=3, padding=1), cplxnn.CplxModReLU(),
    cplxnn.CplxConv1d(384, 256, kernel_size=3, padding=1), cplxnn.CplxModReLU(),
    #nn.Conv1d(256, 256, kernel_size=3, stride=2),
    #nn.Flatten(),
    cplxnn.CplxToCplx[torch.nn.Flatten](start_dim=-2),
    cplxnn.CplxDropout(p=0.5),
    cplxnn.CplxLinear(flatten_dim, 4096), cplxnn.CplxModReLU(),
    cplxnn.CplxDropout(p=0.5),
    cplxnn.CplxLinear(4096, 4096), cplxnn.CplxModReLU(),
    cplxnn.CplxLinear(4096, classes))

seq = nn.Sequential(
    cplxnn.RealToCplx(),
    self.complex_backbone,
    cplxnn.CplxToReal(),
)

NameError: name 'flatten_dim' is not defined